# Managing Agent State - Saving and Loading Agents

In [1]:
import asyncio
from autogen_core import CancellationToken
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient,AzureOpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('AZURE_OPENAI_API_KEY')
model_client = AzureOpenAIChatCompletionClient(
    # azure_deployment="Basavaraj-Bijali",  # This is your deployment name, not the model name
    model="gpt-4o",  # Model name, e.g., "gpt-4"
    api_version="2024-12-01-preview",  # Use the API version your deployment supports
    azure_endpoint="https://tvscs-az-openai.openai.azure.com/",
    api_key=api_key
)

In [2]:
assistant = AssistantAgent(
    name = 'Assistant',
    model_client = model_client,
    system_message = 'You are a helpful assistant who can answer questions and provide information on a wide range of topics. You are friendly, informative, and concise.'
)

response = await assistant.on_messages(
    [TextMessage(content = 'Can you help me with a travel itinerary of France from May to June? Keep it under 30 words.',
    source = 'user')],
    cancellation_token = CancellationToken(),
)

print(response.chat_message)

source='Assistant' models_usage=RequestUsage(prompt_tokens=62, completion_tokens=56) metadata={} content="Sure!  \nMay: Paris (Eiffel Tower, Louvre), Normandy (Mont-Saint-Michel), Loire Valley (châteaux).  \nJune: Provence (lavender fields), Côte d'Azur (Nice, Monaco), Burgundy (vineyards), Lyon (gastronomy)." type='TextMessage'


/Users/basavarajbijali/Desktop/Personal/Udemy_AI_AGENT_COURCE_Jun25/Autogen/autogen_env/lib/python3.12/site-packages/autogen_agentchat/agents/_assistant_agent.py:955: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to gpt-4o-2024-11-20 to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


In [3]:
agent_state = await assistant.save_state()

In [4]:
print(agent_state)

{'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Can you help me with a travel itinerary of France from May to June? Keep it under 30 words.', 'source': 'user', 'type': 'UserMessage'}, {'content': "Sure!  \nMay: Paris (Eiffel Tower, Louvre), Normandy (Mont-Saint-Michel), Loire Valley (châteaux).  \nJune: Provence (lavender fields), Côte d'Azur (Nice, Monaco), Burgundy (vineyards), Lyon (gastronomy).", 'thought': None, 'source': 'Assistant', 'type': 'AssistantMessage'}]}}


In [5]:
new_assistant_agent = AssistantAgent(
    name = 'Assistant_agent_2',
    model_client = model_client,
    system_message = 'You are a helpful assistant.'
)

In [6]:
await new_assistant_agent.load_state(agent_state)

In [7]:

response = await new_assistant_agent.on_messages(
    [TextMessage(content = 'Can you tell me what was the last discussion about?',
    source = 'user')],
    cancellation_token = CancellationToken(),
)
print(response.chat_message)

source='Assistant_agent_2' models_usage=RequestUsage(prompt_tokens=115, completion_tokens=56) metadata={} content="Sure! You asked me to create a travel itinerary for France from May to June, keeping it under 30 words. I suggested exploring Paris, Normandy, Loire Valley, Provence, Côte d'Azur, Burgundy, and Lyon. Let me know if you'd like more details!" type='TextMessage'


In [8]:
response.chat_message.content

"Sure! You asked me to create a travel itinerary for France from May to June, keeping it under 30 words. I suggested exploring Paris, Normandy, Loire Valley, Provence, Côte d'Azur, Burgundy, and Lyon. Let me know if you'd like more details!"